# Traitement de données massives (Big Data) avec PySpark



# Détection d’anomalies (transactions bancaires frauduleuses) = implémenter, entraîner et comparer la performance d'un modèle de Random Forest & de régression logistique avec PySpark (Big Data)

In [1]:
#Import de SparkSession et SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkContext

# Définition d'un SparkContext en local
sc = SparkContext()

# Construction d'une Session Spark
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("PySpark") \
    .getOrCreate()
    
# spark
spark

In [2]:
# Importer le fichier creditcard.csv en tant que DataFrame appelé df_raw
df_raw = spark.read.csv('creditcard.csv', header = True)

# Afficher un extrait du DataFrame df_raw
df_raw.sample(False, .00005, seed = 222).toPandas()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,42234,-1.7524331477332,-0.419055733827287,2.53528644046672,0.0435559315179309,1.19562715748271,2.11740918676437,0.622240351843119,0.0697994996855651,0.76353153361211,...,-0.0286771358898975,0.830596805435572,-0.335279683642818,-1.00779708785447,0.245120949124039,-0.35929465581098,-0.731133628928866,-0.426853189642464,87.11,0
1,42641,-0.873102958309257,-0.466765914550112,2.23215730360367,1.73103360762164,0.652687670394182,0.763353265492849,-0.199816332688368,0.241234378153372,0.803049715750664,...,-0.29981676581135,-0.510785112040975,0.14233434055339,-0.463695297130048,-0.299635035162249,-0.433643172068359,0.0288172443772179,-0.0548318581042295,53.73,0
2,45351,-1.0568688599443,0.38433504372213,0.834099707878581,-1.81454179209205,-0.523873927518483,-0.00663293153702545,-0.686254132914173,-2.70547500574678,-1.89222291887493,...,-0.875309388105664,1.38267916351104,-0.272977007236098,0.0550718778421547,-0.0999609938893566,-0.303686035486177,0.152844671937108,0.120109403842216,121,0
3,54459,-2.88433244607372,0.241184293767801,1.81832594577724,0.315421897263641,-1.2025760719795,2.67407177415621,-0.0892431339912677,0.822561588291796,1.50632427582946,...,0.00861337881566351,0.851009974523584,-0.77548508676568,-1.22096362556006,0.557296647095748,-0.271862775394419,-1.5667425034041,-0.994205275432934,175,0
4,58359,1.27278370614652,0.304278625279372,-0.166548445622582,0.727912098191114,0.398891119781968,0.0403957737828267,0.134915762694388,-0.0436335339438501,-0.279520160859254,...,0.0121653235182436,0.065317283362877,-0.267363016801172,-0.810313134509659,0.829479947069833,-0.275886602794235,0.0108228919465189,0.000520230741261828,10.19,0
5,59506,-0.698542753874564,-0.901166276563582,1.93993316494701,-2.56048267875261,-1.11576281976222,0.0146003358539483,-0.431969272426638,0.242707645872424,-2.36517651930388,...,0.163905394127072,0.562500997743803,0.0763693773047964,-0.0190495450510778,-0.100030450165238,-0.266963186404007,0.0872444750194298,0.107071446115603,100,0
6,90876,2.08420913852485,0.138856485263937,-1.37306551903783,0.416321268401018,0.454360891746392,-0.725923885481793,0.14218697763232,-0.398085729250541,1.76779865170568,...,0.0956043374403373,0.690043578264837,0.0279645534348086,0.597622560831445,0.289559624267704,-0.172944321819026,-0.0440061852271083,-0.0579852531412462,3.59,0
7,114968,1.84135740356269,-0.544824401135573,-0.30169196986034,0.403788537035152,-0.788327910541013,-0.717142550159655,-0.335489668186148,-0.170292870483058,0.928976535144486,...,-0.154559741543096,-0.464456651461227,0.339302235168314,0.0730055051501035,-0.635254971467599,0.258801317209045,-0.0434190846559531,-0.0260956193016201,90,0
8,116219,0.0945009105637923,1.04850600008133,-2.44116990542136,-0.818460333337403,1.67223448074053,-0.206843394301094,1.07972015616618,0.304044056095126,-0.639296951681487,...,0.163164931449877,0.324689875043219,-0.170457969421123,-0.759979546149371,-0.228049101679919,0.581764038415345,-0.113033792379452,0.0161543031855033,42.81,0
9,129508,2.28416903719626,-0.590734810000216,-1.73417405056201,-1.15833260448266,0.11674331940433,-0.584527459730381,-0.263633990664876,-0.386956525664767,-0.742329516511752,...,0.400441470931391,1.2208042807285,-0.083955939522684,0.189027924455621,0.370322117383986,0.0700754264076844,-0.0345092019278939,-0.061934421066604,15,0


J'ai créé un DataFrame df à partir de df_raw en changeant les colonnes des variables cibles en double et la variable cible, Class, en int

Puis j'ai affiché le schéma des variables de df


Comme bonne pratique, j'ai mis la variable à prédire ou la variable cible dans la première colonne


In [3]:
# Import de col du package pyspark.sql.functions
from pyspark.sql.functions import col

# La variable Class est située dans la dernière colonne, on change donc en double les 30 premières colonnes
exprs = [col(c).cast("double") for c in df_raw.columns[:30]]

# On met la variable classe dans la première colonne en la changeant en type int au passage, puis on met les autres colonnes
df = df_raw.select(col(df_raw.columns[30]).cast('int'), *exprs)

# affichage du schéma des variables "df"
df.printSchema()

root
 |-- Class: integer (nullable = true)
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nu

Il n'y a pas de valeurs manquantes dans le dataset

J'affiche le nombre d'individus par modalité de Class

In [4]:
df.groupBy('Class').count().show()

+-----+------+
|Class| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



Je créé un DataFrame df0 contenant uniquement des observations de la Class 0

Il devra y avoir autant d'individus que dans la Class 1 du DataFrame df

In [5]:
df0 = df.filter(df.Class == 0).sample(False, 0.0017, seed = 222)

La méthode unionAll d'un DataFrame renvoie un DataFrame où l'objet appelant la méthode et le paramètre sont mélangés

J'ai crée un DataFrame new_df où les deux Class sont uniforméments distribuées

In [6]:
new_df = df0.unionAll(df.filter(df.Class == 1))
new_df.groupBy('Class').count().show()

+-----+-----+
|Class|count|
+-----+-----+
|    1|  492|
|    0|  473|
+-----+-----+



J'ai créé :

un rdd rdd_ml séparant la variable à expliquer des features (à mettre sous forme DenseVector)

un DataFrame df_ml contenant notre base de données sous deux variables : labels et features

In [7]:
# Import de DenseVector du package pyspark.ml.linalg
from pyspark.ml.linalg import DenseVector

# Création d'un rdd en séparant la variable à expliquer des features
rdd_ml = new_df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Création d'un data frame composé de deux variables : label et features
new_df_ml = spark.createDataFrame(rdd_ml, ['label', 'features'])


Par la suite, j'ai crée :

-deux DataFrames appelés train et test contenant chacun respectivement 80% et 20% des données

-un classificateur Random Forest appelé clf


Puis apprendre le modèle des forêts aléatoires sur l'ensemble d'entraînement

In [8]:
# train and test data 
train, test = new_df_ml.randomSplit([.8, .2], seed= 123)

#
from time import time
t0 = time()

# Import de RandomForestClassifier du package pyspark.ml.classification
from pyspark.ml.classification import RandomForestClassifier

# Création d'une fonction de forêt aléatoires
clf = RandomForestClassifier(labelCol="label", featuresCol="features", predictionCol='prediction')

# Apprentissage des données d'entraînement : "train"
model = clf.fit(train)

print(time() - t0)

# Calcul des prédictions des données de test
predicted = model.transform(test)

# Affichage des prédictions
predicted.show()

6.438539028167725
+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[2012.0,-0.410279...|[18.7202452783780...|[0.93601226391890...|       0.0|
|    0|[2493.0,0.5587202...|[18.6054224644107...|[0.93027112322053...|       0.0|
|    0|[2783.0,-0.359516...|[18.4570982873013...|[0.92285491436506...|       0.0|
|    0|[6010.0,-6.962661...|[16.2034023395441...|[0.81017011697720...|       0.0|
|    0|[7447.0,-5.726395...|[12.9074417403977...|[0.64537208701988...|       0.0|
|    0|[18676.0,-2.15399...|[12.5018825282347...|[0.62509412641173...|       0.0|
|    0|[23663.0,-2.13395...|[18.5900100993534...|[0.92950050496767...|       0.0|
|    0|[26803.0,-1.30667...|[15.3611632354055...|[0.76805816177027...|       0.0|
|    0|[27722.0,-0.80876...|[18.5970989155384...|[0.92985494577692...|       0.0

J'ai calculé la précision (accuracy) du modèle entraîné

In [9]:
# Import d'un évaluateur MulticlassClassificationEvaluator du package pyspark.ml.evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Création d'un évaluateur 
evaluator = MulticlassClassificationEvaluator(metricName='accuracy',
                                              labelCol= 'label',
                                              predictionCol= 'prediction')
# Calcul et affichage de la précision du modèle 
accuracy = evaluator.evaluate(predicted)
print(accuracy)

0.9411764705882353


Par la suite :

- j'ai créé un modèle de régression logistique

- à l'aide de l'évaluateur précédent, cross-validé ce modèle pour le paramètre regParem appartenant à $[0, 0.01, 0.1, 1]$

- j'ai comparé la précision des deux modèles obtenues


NB : Il se peut que la cross validation prenne un peu de temps (~ 2-3 minutes)

In [10]:
# Import de la régression logistique et de la cross-validation ainsi que de la grille de paramètres
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Création de la régression logistique
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label')

# Création de la grille de paramètre
param_grid = ParamGridBuilder().\
    addGrid(lr.regParam, [0, 0.01, 0.1, 1]).\
    build()

# Création du modèle de cross-validation
cv = CrossValidator(estimator = lr, estimatorParamMaps = param_grid, evaluator = evaluator, numFolds = 3)

# Entraînement du modèle précédent
cv_model = cv.fit(train)

# Calcul de la deuxième prédiction et de la précision
predicted2 = cv_model.transform(test)
accuracy2 = evaluator.evaluate(predicted2)

# Affichage des deux précisions
print(accuracy, accuracy2)

0.9411764705882353 0.9558823529411765


Penser à la fin à stopper la session spark :

In [11]:
spark.stop()
sc.stop()